In [1]:
import QuantLib as ql

In [2]:
class DMoneyMarketSwap:
    def __init__(self, evDate, effective, maturity, position, CCYComm, InitNPAComm, 
                 FinalNPAComm, CCYTerm, InitNPATerm, FinalNPATerm, FXSpot, h_discTSComm,
                 h_discTSTerm):
        self._ValueDate = evDate
        self._EffectiveDate = effective
        self._Maturity = maturity
        if position == 'Buy':      
            self._Position = 1.0
        else:
            self._Position = -1.0
        self._CCYComm = CCYComm
        self._InitNPAComm = InitNPAComm
        self._FinalNPAComm = FinalNPAComm
        self._CCYTerm = CCYTerm
        self._InitNPATerm = InitNPATerm
        self._FinalNPATerm = FinalNPATerm
        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm
        
    def NPV(self):
        FinalDfComm = self._H_discTSComm.discount(self._Maturity)
        FinalDfTerm = self._H_discTSTerm.discount(self._Maturity)
        FinalPVComm = FinalDfComm * self._FinalNPAComm
        FinalPVTerm = FinalDfTerm * self._FinalNPATerm
        if (self._ValueDate >= self._EffectiveDate):
            self._Value = self._Position * (FinalPVComm * self._FXSpot - FinalPVTerm)
        else:
            InitDfComm = self._H_discTSComm.discount(self._EffectiveDate)
            InitDfTerm = self._H_discTSTerm.discount(self._EffectiveDate)
            InitPVComm = InitDfComm * self._InitNPAComm
            InitPVTerm = InitDfTerm * self._InitNPATerm
            self._Value = self._Position * ((FinalPVComm * self._FXSpot - FinalPVTerm)
                                            + (InitPVTerm - InitPVComm * self._FXSpot))
        return self._Value

In [3]:
settings = ql.Settings.instance()
evDate = ql.Date(11, 7, 2011)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
DC360 = ql.Actual360()
settlementDays = 0
refDate = Cal.advance(evDate, settlementDays, ql.Days, ql.Following, False)
BDC = ql.Unadjusted

In [4]:
f = 0.01
discTSComm = ql.FlatForward(settlementDays, Cal, f, DC360, ql.Continuous, ql.Annual)
h_discTSComm = ql.YieldTermStructureHandle(discTSComm)

In [5]:
r = 0.02
discTSTerm = ql.FlatForward(settlementDays, Cal, r, DC365, ql.Continuous, ql.Annual)
h_discTSTerm = ql.YieldTermStructureHandle(discTSTerm)

In [6]:
effective = ql.Date(13, 7, 2011)
maturity = ql.Date(13, 1, 2012)
position = 'Buy'
CCYComm = 'USD'
InitCommNPA = 1000000
FinalCommNPA = 1000000
CCYTerm = 'TWD'
InitTermNPA = 28000000
FinalTermNPA = 27900000
FXSpot = 28.0

In [7]:
MMS = DMoneyMarketSwap(evDate, effective, maturity, position, CCYComm, InitCommNPA, 
                       FinalCommNPA, CCYTerm, InitTermNPA, FinalTermNPA, FXSpot, 
                       h_discTSComm, h_discTSTerm)
Value = MMS.NPV()
print(Value)

237100.1752630137
